# Business Understanding 

CMS rates hospitals in the US on a scale of 1-5 with the objective to make it easier for patients and consumers to compare the quality of hospitals.
The ratings directly influence the choice of the hospital made by consumers and may have a significant impact on the revenue earned by hospitals.
Thus, it is extremely important for hospitals to understand the methodology used by CMS for calculating the ratings so that they can work on
improving the factors that influence them.
 
This project is focused on developing an approach to calculate hospital ratings and using it to identify areas of improvement for
certain hospitals. It will also require a thorough understanding of the rating system developed by CMS.


# Business Problem 

The aim of analysis is to understand the methodology used by CMS for calculating the ratings and identify
the factors influencing the ratings for hospitals, so that they can work on improving the factors that influence them.
Recommend ways to improve the rating for Evanston Hospital to improve their current star rating of 3/5*



# Data Understanding #

The original source of data is `Hospital_Revised_FlatFiles_20161110`
  CSV Files
1.	Readmissions and Deaths - Hospital.csv	                                                readmission.csv
2.	Readmissions and Deaths - Hospital.csv +   Complications - Hospital.csv	                mortality.csv
3.	Healthcare Associated Infections - Hospital.csv +   Complications - Hospital.csv	      safety.csv
4.	HCAHPS - Hospital.csv	                                                                  experience.csv
5.	Outpatient Imaging Efficiency - Hopital.csv	                                            medical.csv
6.	Timely and Effective Care - Hospital.csv	                                              timeliness.csv
7.	Timely and Effective Care - Hospital.csv	                                              effectiveness.csv


# Exploratory Data Analysis 

- Perform the Univariate analysis for all the groups.
- Perform Bi-variate analysis for all the groups.


Modelling 

 Part 1 - Supervised Learning-Based Rating
 Part 2 - Factor analysis and Clustering-Based Rating (Unsupervised)
 Part 3 - Provider analysis - Recommendations for Hospitals

Let us load the data and create the groups as above:
Copied the required raw files to the Groups location.
1. "Readmissions and Deaths - Hospital.csv"
2. "Complications - Hospital.csv"
3. "Healthcare Associated Infections - Hospital.csv"
4. "HCAHPS - Hospital.csv"
5. "Outpatient Imaging Efficiency - Hospital.csv"
6. "Timely and Effective Care - Hospital.csv"



# Data Prepartion, cleaning and Supervised Modelling 
Data set contains 58 excel files, 2 PDF files out of this for this assignment we require 6 files & it has suffix as "_Hospital"

Load the files into dataframes:
1. Load the data - replace Not Available, Not Applicable with NA  (Suffix _Raw dataframes )
2. Split xxxx_rawdata frames into 2 data frames [xxxx_hosp, xxxx_meas)
3. Rename columns - Standardize names across
4. Reorder columns to match all files
5. Standardize the measures- some measures with positive zscores and some measures with negative zscores.
6. Impute the outliers

# Libraries to be used


In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_context('paper')



# Utility functions 

In [2]:
def print_ln():
    print('-' * 80, '\n')


# Function to create a subset of the dataframe
def subset(dataframe, col_name, col_names_list):
    return dataframe.loc[dataframe[col_name].isin(col_names_list)]

# Converts the datatype of a specific column and returns the new dataframe
def func_numeric(df, col_name):
    df[col_name] = df[col_name].astype(float)
    return df

# Renames a column in the dataframe by appending `_score`.
def func_rename(df):
    old_col_names = df.columns.to_list()
    new_col_names = []
    for a_col_name in old_col_names:
        col_name = a_col_name + "_score"
        new_col_names.append(col_name)

    name_pairs = dict(zip(old_col_names, new_col_names))
    df = df.rename(columns=name_pairs)
    return df

# Function to compute the negative zscore value for the dataframe

def negative_zscore(dataframe):
    df = dataframe.copy()
    cols = list(df.columns)
    for col in cols:
        df[col] = - (df[col] - df[col].mean())/df[col].std(ddof=0)
    return df

# Function to compute the positive zscore value for the dataframe
def positive_zscore(dataframe):
    df = dataframe.copy()
    cols = list(df.columns)
    for col in cols:
        df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)
    return df

# Function to compute the valid subset of a dataframe i.e. reduces outliers via IQR method
def subset_by_iqr(df, column, whisker_width=0):
    # Calculate Q1, Q2 and IQR
    q1 = df[column].quantile(0.00125)
    q3 = df[column].quantile(0.99875)
    iqr = q3 - q1
    # Apply filter with respect to IQR, including optional whiskers
    filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
    return df.loc[filter][column]


# Driver function for `subset_by_iqr` which treats the outliers in a dataframe
def treat_outliers(dataframe):
    df = dataframe.copy()
    cols = list(df.columns)
    for col in cols:
        df[col] = subset_by_iqr(df, col)
    return df

In [3]:

# Computes the group score of a dataframe after cleaning and doing PCA on the group of measures
def function_group_score(numeric_df, score_name):
    # CMS recommends atleast 3 non-null measures per group
    df = numeric_df.dropna(thresh= 3)
    imputed_df = df.apply(lambda x: x.fillna(x.median()), axis=0)
    pca = IncrementalPCA()
    df_pca = pca.fit_transform(imputed_df)
    df_pca = pd.DataFrame(df_pca, columns= df.columns)
    df_pca.index = df.index
    df_with_weight = df_pca.mean(axis=1)
    df_scores = pd.DataFrame({score_name : df_with_weight})
    return df_scores

## 1. Readmission - Load "Readmissions and Deaths - Hospital.csv" file into read_raw

In [5]:
# Readmissions and deaths 

read_rawdata = pd.read_csv("Readmissions and Deaths - Hospital.csv",
                           encoding="ISO-8859-1",
                           na_values=["Not Available", "Not Applicable"])

# We will filter only those columns which are needed 
read_meas_list =   ["READM_30_AMI", "READM_30_CABG", "READM_30_COPD", "READM_30_HF", "READM_30_HIP_KNEE", "READM_30_HOSP_WIDE", "READM_30_PN", "READM_30_STK"]

read_hosp = read_rawdata.iloc[:,0:8]
read_hosp.head()


,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701
4,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701


In [6]:
# Drop duplicates
read_hosp = read_hosp.drop_duplicates(keep='first')

In [7]:
# Select the relevant columns
read_meas = read_rawdata.iloc[: , [0,9,12]]
read_meas.head()

,Provider ID,Measure ID,Score
0,10001,MORT_30_AMI,12.5
1,10001,MORT_30_CABG,4.2
2,10001,MORT_30_COPD,9.3
3,10001,MORT_30_HF,12.4
4,10001,MORT_30_PN,15.5


In [8]:
# Converting the datatype of Score to a float

read_meas['Score'] = read_meas['Score'].astype(float)
read_meas.head()


,Provider ID,Measure ID,Score
0,10001,MORT_30_AMI,12.5
1,10001,MORT_30_CABG,4.2
2,10001,MORT_30_COPD,9.3
3,10001,MORT_30_HF,12.4
4,10001,MORT_30_PN,15.5


In [9]:
read_meas = read_meas.loc[read_meas['Measure ID'].isin(read_meas_list)]
read_meas.head()


,Provider ID,Measure ID,Score
6,10001,READM_30_AMI,16.5
7,10001,READM_30_CABG,15.1
8,10001,READM_30_COPD,21.1
9,10001,READM_30_HF,21.4
10,10001,READM_30_HIP_KNEE,5.1


In [10]:
# Restructure the dataframe to better suit a shape for our analysis
read_meas_score = read_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')
read_meas_score.head()


Measure ID,READM_30_AMI,READM_30_CABG,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK
Provider ID,,,,,,,,
10001,16.5,15.1,21.1,21.4,5.1,15.4,18.7,12.7
10005,16.7,NaN,18.0,21.9,5.7,14.9,16.4,13.4
10006,16.1,15.2,19.8,20.6,5.0,15.4,17.9,12.0
10007,NaN,NaN,19.9,21.1,NaN,16.6,17.3,12.7
10008,NaN,NaN,19.2,23.1,NaN,15.7,16.0,NaN


In [11]:
read_meas_score =  func_rename(read_meas_score)
read_meas_score.head()

Measure ID,READM_30_AMI_score,READM_30_CABG_score,READM_30_COPD_score,READM_30_HF_score,READM_30_HIP_KNEE_score,READM_30_HOSP_WIDE_score,READM_30_PN_score,READM_30_STK_score
Provider ID,,,,,,,,
10001,16.5,15.1,21.1,21.4,5.1,15.4,18.7,12.7
10005,16.7,NaN,18.0,21.9,5.7,14.9,16.4,13.4
10006,16.1,15.2,19.8,20.6,5.0,15.4,17.9,12.0
10007,NaN,NaN,19.9,21.1,NaN,16.6,17.3,12.7
10008,NaN,NaN,19.2,23.1,NaN,15.7,16.0,NaN


In [12]:
# Let's see what is the sum of all values of a measure, counting NaN as a 0
print(np.nansum(read_meas_score['READM_30_AMI_score'].unique()))



1104.3


In [13]:
# We will use negative zscore scaling as high readmissions implies the Hospital is not doing well in terms of patient treatment quality.

readmission = negative_zscore(read_meas_score)
readmission.head()


Measure ID,READM_30_AMI_score,READM_30_CABG_score,READM_30_COPD_score,READM_30_HF_score,READM_30_HIP_KNEE_score,READM_30_HOSP_WIDE_score,READM_30_PN_score,READM_30_STK_score
Provider ID,,,,,,,,
10001,0.408730,-0.615156,-0.866907,0.370423,-0.884170,0.215408,-1.105482,-0.124569
10005,0.199974,NaN,1.574551,0.036453,-1.968440,0.820649,0.495640,-0.777118
10006,0.826243,-0.703937,0.156930,0.904776,-0.703459,0.215408,-0.548570,0.527980
10007,NaN,NaN,0.078174,0.570805,NaN,-1.237171,-0.130886,-0.124569
10008,NaN,NaN,0.629471,-0.765075,NaN,-0.147737,0.774096,NaN


In [14]:
# Outlier treatment: According to the CMS documentation, they've performed the outlier treatment for 
# measures at the 0.125th and the 99.875th percentiles
readmission = treat_outliers(readmission)
readmission.head()


Measure ID,READM_30_AMI_score,READM_30_CABG_score,READM_30_COPD_score,READM_30_HF_score,READM_30_HIP_KNEE_score,READM_30_HOSP_WIDE_score,READM_30_PN_score,READM_30_STK_score
Provider ID,,,,,,,,
10001,0.408730,-0.615156,-0.866907,0.370423,-0.884170,0.215408,-1.105482,-0.124569
10005,0.199974,NaN,1.574551,0.036453,-1.968440,0.820649,0.495640,-0.777118
10006,0.826243,-0.703937,0.156930,0.904776,-0.703459,0.215408,-0.548570,0.527980
10007,NaN,NaN,0.078174,0.570805,NaN,-1.237171,-0.130886,-0.124569
10008,NaN,NaN,0.629471,-0.765075,NaN,-0.147737,0.774096,NaN


In [15]:

readmission.to_csv("cleaned_readmission_data.csv")

readmission.head()




Measure ID,READM_30_AMI_score,READM_30_CABG_score,READM_30_COPD_score,READM_30_HF_score,READM_30_HIP_KNEE_score,READM_30_HOSP_WIDE_score,READM_30_PN_score,READM_30_STK_score
Provider ID,,,,,,,,
10001,0.408730,-0.615156,-0.866907,0.370423,-0.884170,0.215408,-1.105482,-0.124569
10005,0.199974,NaN,1.574551,0.036453,-1.968440,0.820649,0.495640,-0.777118
10006,0.826243,-0.703937,0.156930,0.904776,-0.703459,0.215408,-0.548570,0.527980
10007,NaN,NaN,0.078174,0.570805,NaN,-1.237171,-0.130886,-0.124569
10008,NaN,NaN,0.629471,-0.765075,NaN,-0.147737,0.774096,NaN


In [16]:
read_master = pd.merge(read_hosp, readmission, on="Provider ID")
read_master.to_csv("read_master_data.csv")

read_master


,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number,READM_30_AMI_score,READM_30_CABG_score,READM_30_COPD_score,READM_30_HF_score,READM_30_HIP_KNEE_score,READM_30_HOSP_WIDE_score,READM_30_PN_score,READM_30_STK_score
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,3347938701,0.408730,-0.615156,-0.866907,0.370423,-0.884170,0.215408,-1.105482,-0.124569
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,2565938310,0.199974,NaN,1.574551,0.036453,-1.968440,0.820649,0.495640,-0.777118
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,2567688400,0.826243,-0.703937,0.156930,0.904776,-0.703459,0.215408,-0.548570,0.527980
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,3344933541,NaN,NaN,0.078174,0.570805,NaN,-1.237171,-0.130886,-0.124569
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,3343353374,NaN,NaN,0.629471,-0.765075,NaN,-0.147737,0.774096,NaN
5,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,JEFFERSON,2058383122,-0.843809,-0.526375,0.314444,1.439128,-0.884170,0.336456,0.286798,1.180529
6,10012,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,DE KALB,2568453150,0.826243,NaN,0.629471,-0.965457,NaN,0.336456,-1.592780,-0.217790
7,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,SHELBY,2056208100,-0.843809,-1.236623,1.653308,0.303629,-0.703459,1.062745,0.008342,0.155095
8,10018,CALLAHAN EYE HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10019,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660,COLBERT,2563864556,NaN,NaN,-2.363284,-0.297517,0.922946,-0.389833,-0.827026,-0.870339


# 2. Mortality - Load 2 Files "Readmissions and Deaths - Hospital.csv + Complications - Hospital.csv" into morality dataframe


In [ ]:
mort_rawdata1 = read_rawdata
mort_rawdata2 = pd.read_csv("Complications - Hospital.csv", encoding="ISO-8859-1", na_values=["Not Available", "Not Applicable"])
mort_rawdata = pd.concat([mort_rawdata1, mort_rawdata2])
mort_meas_list =   ["MORT_30_AMI", "MORT_30_CABG", "MORT_30_COPD", "MORT_30_HF", "MORT_30_PN", "MORT_30_STK", "PSI_4_SURG_COMP"]
mort_hosp = mort_rawdata.iloc[:,0:8]
mort_hosp = mort_hosp.drop_duplicates(keep='first')
mort_meas = mort_rawdata.iloc[: , [0,9,12]]
mort_meas = mort_meas.loc[mort_meas['Measure ID'].isin(mort_meas_list)]
mort_meas = func_numeric(mort_meas, 'Score')
mort_meas_score = mort_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')
mort_meas_score = func_rename(mort_meas_score)
mortality = mort_meas_score
print(mortality)

# Mortality indicates the death rate, higher the number worser is the hospital or provider.
# Since it is related to death rate we will use negative z-score formula.
mortality = negative_zscore(mortality)


mortality = treat_outliers(mortality)
mortality.to_csv("cleaned_mortality_data_py.csv")

mort_master = pd.merge(mort_hosp, mortality, on="Provider ID")
mort_master.to_csv("mort_master_data_py.csv")


## 3. Safety - Load 2 files "Healthcare Associated Infections - Hospital.csv + Complications - Hospital.csv" into safety dataframe


In [ ]:
safe_rawdata1 = mort_rawdata
safe_rawdata2 = pd.read_csv("Healthcare Associated Infections - Hospital.csv", encoding="ISO-8859-1", na_values=["Not Available", "Not Applicable"])
safe_rawdata1 = safe_rawdata1.iloc[: , [0,1, 2, 3, 4, 5, 6, 7, 9, 12]]
safe_rawdata2 = safe_rawdata2.iloc[: , [0,1, 2, 3, 4, 5, 6, 7, 9, 11]]
safe_rawdata = pd.concat([safe_rawdata1, safe_rawdata2])
safe_meas_list =   ["HAI_1_SIR", "HAI_2_SIR", "HAI_3_SIR", "HAI_4_SIR", "HAI_5_SIR", "HAI_6_SIR", "COMP_HIP_KNEE", "PSI_90_SAFETY"]
safe_hosp = safe_rawdata.iloc[:,0:8]
safe_hosp = safe_hosp.drop_duplicates(keep='first')
safe_meas = safe_rawdata.iloc[: , [0,8,9]]
safe_meas = safe_meas.loc[safe_meas['Measure ID'].isin(safe_meas_list)]
safe_meas = func_numeric(safe_meas, 'Score')

safe_meas_score = safe_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')
safe_meas_score = func_rename(safe_meas_score)
safety = safe_meas_score

# The HAI measures are related to infections contracted by the patients during their stay in the hospital
# we will negative zscore here as well
safety = negative_zscore(safety)
safety = treat_outliers(safety)
safety.to_csv("cleaned_safety_data_py.csv")
safety
safe_master = pd.merge(mort_hosp, safety, on="Provider ID")
safe_master.to_csv("safe_master_data_py.csv")

## 4. Experience - Load file "HCAHPS - Hospital.csv" into experience data rame

In [ ]:

expe_rawdata = pd.read_csv("HCAHPS - Hospital.csv", encoding="ISO-8859-1",
                           na_values=["Not Available", "Not Applicable"])


expe_new_col_names = {"HCAHPS Question": "Measure Name",
                      "HCAHPS Measure ID": "Measure ID",
                      "HCAHPS Linear Mean Value": "Score"}

expe_rawdata = expe_rawdata.rename(columns=expe_new_col_names)



expe_meas_list = ["H_CLEAN_LINEAR_SCORE", "H_COMP_1_LINEAR_SCORE", "H_COMP_2_LINEAR_SCORE", "H_COMP_3_LINEAR_SCORE",
                  "H_COMP_4_LINEAR_SCORE", "H_COMP_5_LINEAR_SCORE", "H_COMP_6_LINEAR_SCORE", "H_COMP_7_LINEAR_SCORE",
                  "H_HSP_RATING_LINEAR_SCORE", "H_QUIET_LINEAR_SCORE", "H_RECMND_LINEAR_SCORE"]


expe_hosp = expe_rawdata.iloc[:, 0:8]



expe_hosp = expe_hosp.drop_duplicates(keep='first')


expe_meas = expe_rawdata.iloc[:, [0, 8, 15]]


expe_meas = expe_meas.loc[expe_meas['Measure ID'].isin(expe_meas_list)]


expe_meas = func_numeric(expe_meas, 'Score')


expe_meas_score = expe_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')


expe_meas_score = func_rename(expe_meas_score)


experience = expe_meas_score


# It measures cleanliness, patient hospitality and doctors/staff communication,
# hospital environment etc. We will use positive zscore here
experience = positive_zscore(experience)

experience = treat_outliers(experience)
experience.to_csv("cleaned_experience_data_py.csv")


expe_master = pd.merge(expe_hosp, experience, on="Provider ID")
expe_master.to_csv("expe_master_data_py.csv")


# 5. Medical - Load file "Outpatient Imaging Efficiency - Hopital.csv" into medical data frame


In [ ]:
medi_rawdata = pd.read_csv("Outpatient Imaging Efficiency - Hospital.csv", encoding="ISO-8859-1",
                           na_values=["Not Available", "Not Applicable"])

medi_meas_list = ["OP_10", "OP_11", "OP_13", "OP_14", "OP_8"]

medi_hosp = medi_rawdata.iloc[:, 0:8]

medi_hosp = medi_hosp.drop_duplicates(keep='first')
# medi_hosp
# print_ln()
# medi_hosp.dtypes

medi_meas = medi_rawdata.iloc[:, [0, 8, 10]]
# medi_meas
# print_ln()

medi_meas = subset(medi_meas, 'Measure ID', medi_meas_list)
# medi_meas
# medi_meas.dtypes
# print_ln()

medi_meas = func_numeric(medi_meas, 'Score')
# medi_meas.dtypes
# print_ln()


medi_meas_score = medi_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')
# medi_meas_score
# print_ln()

medi_meas_score = func_rename(medi_meas_score)
# medi_meas_score
# print_ln()


medical = medi_meas_score
# medical
# print_ln()

# Unecessary usage of imaging tests, lower the better. We will use the negative zscore

medical = negative_zscore(medical)
# medical
# print_ln()

medical = treat_outliers(medical)
# medical
# print_ln()

medi_master = pd.merge(medi_hosp, medical, on="Provider ID")
# medi_master
# print_ln()

medical.to_csv("cleaned_medical_data_py.csv")

# 6. Timeliness - Load file "Timely and Effective Care - Hospital.csv" into timeliness data frame


In [ ]:
time_rawdata = pd.read_csv("Timely and Effective Care - Hospital.csv", encoding="ISO-8859-1",
                           na_values=["Not Available", "Not Applicable"])
# time_rawdata
# print_ln()

time_meas_list = ["ED_1b", "ED_2b", "OP_18b", "OP_20", "OP_21", "OP_3b", "OP_5"]

time_hosp = time_rawdata.iloc[:, 0:8]
# time_hosp
# print_ln()
# time_hosp.dtypes


time_hosp = time_hosp.drop_duplicates(keep='first')
# time_hosp
# print_ln()
# time_hosp.dtypes

time_meas = time_rawdata.iloc[:, [0, 9, 11]]
# time_meas
# print_ln()


time_meas = subset(time_meas, 'Measure ID', time_meas_list)
# time_meas
# time_meas.dtypes
# print_ln()

time_meas = func_numeric(time_meas, 'Score')
# time_meas.dtypes
# print_ln()

time_meas_score = time_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')
# time_meas_score
# print_ln()

time_meas_score = func_rename(time_meas_score)
# time_meas_score
# print_ln()

timeliness = time_meas_score
# timeliness
# print_ln()

# All the measures in timeliness indicate the average time the patient had to wait
# before being attended by the doctors or concerned specialists. We will use negative zscore

timeliness = negative_zscore(timeliness)
# timeliness
# print_ln()

timeliness = treat_outliers(timeliness)
# timeliness
# print_ln()

time_master = pd.merge(time_hosp, timeliness, on="Provider ID")
# time_master
# print_ln()

timeliness.to_csv("cleaned_timeliness_data_py.csv")


## 7. Effectiveness - Load file "Timely and Effective Care - Hospital.csv" into timeliness data frame


In [ ]:

effe_rawdata = time_rawdata
# effe_rawdata
# print_ln()

effe_meas_list = ["CAC_3", "IMM_2", "IMM_3_OP_27_FAC_ADHPCT", "OP_22", "OP_23", "OP_29", "OP_30", "OP_4", "PC_01", "STK_4", "STK_5", "STK_6", "STK_8", "VTE_1", "VTE_2", "VTE_3", "VTE_5", "VTE_6"]

effe_hosp = effe_rawdata.iloc[:, 0:8]
# effe_hosp
# print_ln()
# effe_hosp.dtypes


effe_hosp = effe_hosp.drop_duplicates(keep='first')
# effe_hosp
# print_ln()
# effe_hosp.dtypes

effe_meas = effe_rawdata.iloc[:, [0, 9, 11]]
# effe_meas
# print_ln()


effe_meas = subset(effe_meas, 'Measure ID', effe_meas_list)
# effe_meas
# effe_meas.dtypes
# print_ln()

effe_meas = func_numeric(effe_meas, 'Score')
# effe_meas.dtypes
# print_ln()

effe_meas_score = effe_meas.pivot(index='Provider ID', columns='Measure ID', values='Score')
# effe_meas_score
# print_ln()


effe_meas_score = func_rename(effe_meas_score)
# effe_meas_score
# print_ln()

effectiveness = effe_meas_score
# effectiveness
# print_ln()


# Effectiveness has some columns for which the value higher is better, and few the score lower is better
# We will both postive and negative zscores here for the filtered columns

positive_measures = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]
effectiveness.iloc[:, positive_measures] = positive_zscore(effectiveness.iloc[:, positive_measures])
# effectiveness.iloc[:, positive_measures].columns
# print_ln()

negative_measures = [3, 8, 17]
effectiveness.iloc[:, negative_measures] = negative_zscore(effectiveness.iloc[:, negative_measures])
# effectiveness.iloc[:, negative_measures].columns
# print_ln()

effectiveness = treat_outliers(effectiveness)
# effectiveness
# print_ln()

effe_master = pd.merge(effe_hosp, effectiveness, on="Provider ID")
# effe_master
# print_ln()

effectiveness.to_csv("cleaned_effectiveness_data_py.csv")



In [ ]:

merge1 = pd.merge(read_master, mort_master)
merge2 = pd.merge(merge1, safe_master)
merge3 = pd.merge(merge2, expe_master)
merge4 = pd.merge(merge3, medi_master)
merge5 = pd.merge(merge4, time_master)
merge6 = pd.merge(merge5, effe_master)

print(merge6.columns.to_list)
print_ln()

print(merge6)
print_ln()

master_data_x = merge6

# Raw data is ready with all required 64 measures and 8 general columns 
master_data_x.to_csv("cleaned_master_data_x_py.csv")


# 2. Data Cleaning

In [ ]:
## Remove duplicate Data

master_data_x = master_data_x.drop_duplicates(keep='first')

master_data_x
print_ln()

master_data_x.isnull().sum()

columns_with_missing_data = round(100 * (master_data_x.isnull().sum() / len(master_data_x.index)), 2)
columns_with_missing_data[columns_with_missing_data > 30].plot(kind='bar')
plt.show()


In [ ]:

 np.sum(master_data_x.isnull().sum().to_list())
print_ln()

## The Score columns are having 131127 NA values, score columns are very important for our further analysis. As these are all independent(X) variables
## we will deal with the NA cleaning after merging with Dependent variable (ratings- y) in later phase

master_data_x



# Load the dependent variable i.e. the `rating` from the  `Hospital Genral Information.csv` file


In [ ]:
hospital_ratings = pd.read_csv("Hospital General Information.csv", encoding="ISO-8859-1", na_values=["Not Available", "Not Applicable"])
hospital_ratings.columns
print_ln()
print(hospital_ratings)
print_ln()

master_data_y = hospital_ratings

hospital_ratings[['Hospital overall rating']].isnull().sum()

master_data_y = master_data_y.iloc[:,[0, 12]]

In [ ]:
print(master_data_y.columns)
print_ln()

## Merge X (independent) & Y (dependent) variables to one using Provider id

In [ ]:
master_data = pd.merge(master_data_x, master_data_y, on= "Provider ID")
print(master_data)
print_ln()


In [ ]:
master_data_without_na = master_data[master_data['Hospital overall rating'].notnull()]
master_data_without_na
print_ln()

print(master_data_without_na)

master_data_with_na = master_data[~master_data['Hospital overall rating'].notnull()]
print(master_data_with_na)
print_ln()

np.sum(master_data_without_na.isnull().sum().to_list())


In [ ]:
columns_with_missing_data = round(100 * (master_data_without_na.isnull().sum() / len(master_data_without_na.index)), 2)


# Remove all columns having more than 50% NA in the dataset .. which are not going to yield any outcome
# which is close to 1824 NA values in any x measure.. will remove the measure
## The following measures having >50% of its data as NA
## READM_30_CABG - 2623 | PSI_4_SURG_COMP - 1831 | HAI_4_SIR - 2804 | HAI_5_SIR - 1860 | OP_3b - 3267 |
## STK_4 - 2762 | STK_5 - 2133 | VTE_6 - 2417

output = columns_with_missing_data[columns_with_missing_data < 50]
output = list(output.to_dict().keys())
print_ln()

master_data_without_na = master_data_without_na[output]
master_data_without_na

In [ ]:
master_data_without_na.isnull().sum()

# Impute only relevant numerical columns
master_data_without_na.iloc[ : , 8:] =  master_data_without_na.iloc[ : , 8:].apply(lambda x: x.fillna(x.median()), axis=0)

master_data_without_na

master_data_without_na = master_data_without_na.drop(master_data_without_na.iloc[:, 1:8],  axis = 1)
master_data_without_na['Hospital overall rating'], factors_hospital_overall_rating =  pd.factorize(master_data_without_na['Hospital overall rating'])
cleaned_master_data = master_data_without_na


cleaned_master_data.to_csv("cleaned_master_data_py.csv")

cleaned_master_data
